### 0. Load environment variables
#### Case 1) If you use OpenAI's ChatGPT, then you need store the following variables in '.env' file:

- 'OPENAI_API_KEY' = XXXXXXXXXXXXXXX
- 'model_name' = XXXXXXXXXXXXXXX

##### Case 2) If you use AzureOpenAI's ChatGPT, then you need store the following variables in '.env' file:

- 'OPENAI_API_TYPE' = 'azure'
- 'OPENAI_API_VERSION' = '2023-08-01-preview'
- 'OPENAI_API_BASE' = XXXXXXXXXXXXXXX
- 'OPENAI_API_KEY' = XXXXXXXXXXXXXXX
- 'deployment_name' = XXXXXXXXXXXXXXX

In [4]:
import os
import sys  
from dotenv import load_dotenv
sys.path.extend(["..", '../..'])  
_ = load_dotenv('.env')


model_name = os.getenv('deployment_name') if os.getenv('OPENAI_API_TYPE') == 'azure' else os.getenv('model_name')
if os.getenv('OPENAI_API_TYPE') == 'azure':
    deployment_name = model_name

### 1. Get a random persona

In [5]:
from utils.pydantic_models.user_persona import UserPersona, get_random_user_persona
persona = get_random_user_persona().json()
persona

'{"gender": "male", "name": "William Short", "language": "English", "nationality": "Netherlands", "age": 89, "hobbies": ["hiking", "painting", "gardening", "pottery"], "talkative": false, "characteristics": ["unknown", "NotJovial", "Vibrant"], "education_level": "undergraduate_college_level"}'

### 2. Get OpenAI functions
This prompt is supposed to work with OpenAI Functions 

In [6]:
from utils.pydantic_models.user_persona import UserProfile
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
openai_function_profile = convert_pydantic_to_openai_function(UserProfile)

### You can also get openai_function_profile in this way
# with open('openai_function_profile.json', 'r') as f:  
#     openai_function_profile = json.load(f)

### 3. Fetch the prompt and create a chain (using LCEL)


In [7]:
from typing import Literal
from langchain import hub
from langchain.chat_models import ChatOpenAI, AzureChatOpenAI

def get_OpenAI_chain(provider: Literal['ChatOpenAI', 'AzureChatOpenAI'], model_or_deployment_name: str):
    if provider == 'ChatOpenAI':
        chat_model = ChatOpenAI(model_name=model_or_deployment_name, temperature=0).bind(functions=[openai_function_profile], function_call={'name':'UserProfile'})
    else:
        chat_model = AzureChatOpenAI(deployment_name=model_or_deployment_name, temperature=0).bind(functions=[openai_function_profile], function_call={'name':'UserProfile'})
    return chat_model

prompt = hub.pull("jet-taekyo-lee/persona-description")
provider = 'AzureChatOpenAI'
model_or_deployment_name = os.getenv('model_name') if provider == 'ChatOpenAI' else os.getenv('deployment_name')
chat_model = get_OpenAI_chain(provider, model_or_deployment_name)

chain = prompt | chat_model

### 4. Get a response

In [8]:
response = chain.invoke({'persona':persona})
print(response.additional_kwargs['function_call']['arguments'])

{
  "original_persona": {
    "gender": "male",
    "name": "William Short",
    "language": "English",
    "nationality": "Netherlands",
    "age": 89,
    "hobbies": ["hiking", "painting", "gardening", "pottery"],
    "talkative": false,
    "characteristics": ["unknown", "NotJovial", "Vibrant"],
    "education_level": "undergraduate_college_level"
  },
  "refined_persona": {
    "gender": "male",
    "name": "William Short",
    "language": "English",
    "nationality": "Netherlands",
    "age": 89,
    "hobbies": ["hiking", "painting", "gardening", "pottery"],
    "talkative": false,
    "characteristics": ["NotJovial", "Vibrant"],
    "education_level": "undergraduate_college_level"
  },
  "refined_persona_description": {
    "description_of_persona": "William Short is an 89-year-old male from the Netherlands. He is a college graduate and is fluent in English. Despite his age, he is quite active and enjoys hobbies such as hiking, painting, gardening, and pottery. He is not very ta